###### cabi.org has Datasheet for more than 10K species, and distribution map is not availiable for some species.  The code here checks the availability of distribution map for all species on CABI that has a datasheet. 

In [1]:
# Load libraries
import pandas as pd
import numpy as np
import os
import requests
from bs4 import BeautifulSoup
import multiprocessing
from joblib import Parallel, delayed
import time

###### The file loaded below was downloaded from CABI.org, with links to all datasheets available.

In [4]:
os.chdir("C:/Users/wliang5/Desktop/cabi")

dt=pd.read_csv('urlCABIsheet.csv',sep=',',header=0,encoding = "ISO-8859-1")
dt.shape
dt.head()

# create a new column "DistributionMap", and its value will be signed as 1 for species with distribution map available
dt["DistributionMap"]=0
display(dt.head())
dt.shape

,Scientific name,Common name,Coverage,URL,DistributionMap
0,Daphnia longispina,NaN,Full,https://www.cabi.org/isc/datasheet/86427,0
1,Anaplasma centrale,NaN,Full,https://www.cabi.org/isc/datasheet/94680,0
2,Anaplasma mesaeterum,NaN,Full,https://www.cabi.org/isc/datasheet/94681,0
3,Homeria,NaN,Basic,https://www.cabi.org/isc/datasheet/101541,0
4,Hucho,NaN,Basic,https://www.cabi.org/isc/datasheet/101560,0


(10868, 5)

###### Define a function to check availablity of distribution map. The function only takes i as input to make parallel processing possible later.

In [5]:
def mapCheck2(i):
    
    import requests
    from bs4 import BeautifulSoup

    url=dt["URL"][i]
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    maps = soup.find(id='toDistributionMaps')

    try:
        n=len(maps)
    except :
        n=-1

    if n!= -1:
        dt.loc[i,"DistributionMap"]=1
    return dt["DistributionMap"][i]
        

###### Set up parallels and record processing time. Just for illustration purpose, only run the function for the first 100 datasheet. I guess this code can be changed to find other available information on the websites. 

In [6]:
ncores=multiprocessing.cpu_count()-2

start=time.time()

re=Parallel(n_jobs=ncores)(delayed(mapCheck2)(i) for i in np.arange(100))

end=time.time()
end-start

#np.savetxt("CabiDistributionMap.csv",re,  delimiter=', ',header="'DistributionMap'")

20.767117738723755

###### 21s, pretty quick.